<a href="https://colab.research.google.com/github/fabiobento/dnn-course-2024-1/blob/main/00_course_folder/cert_prof_dl_intro/2%20-%20Introdu%C3%A7%C3%A3o%20%C3%A0%20vis%C3%A3o%20computacional/14%20-%20C1_W2_Lab_2_callbacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

adaptado de [Certificado Profissional Desenvolvedor do TensorFlow para DeepLearning.AI](https://www.coursera.org/professional-certificates/tensorflow-in-practice) de [Laurence Moroney](https://laurencemoroney.com/)

# Usando Callbacks para Controlar o Treino

Neste laboratório, você usará a [API de Callbacks API](https://keras.io/api/callbacks/) para interromper o treinamento quando uma métrica especificada for atingida.

Esse é um recurso útil para que você não precise concluir todas as épocas quando esse limite for atingido.
> Por exemplo, se você definir 1000 épocas e a precisão desejada já for atingida na época 200, o treinamento será automaticamente interrompido.

Vamos ver como isso é implementado nas próximas seções.

## Carregar e normalizar o conjunto de dados Fashion MNIST

Como no laboratório anterior, você usará o conjunto de dados Fashion MNIST novamente para este exercício.

E também, como mencionado anteriormente, você normalizará os valores de pixel para ajudar a otimizar o treinamento.

In [ ]:
import tensorflow as tf

# Instanciar a API do conjunto de dados
fmnist = tf.keras.datasets.fashion_mnist

# Carregar o conjunto de dados
(x_train, y_train),(x_test, y_test) = fmnist.load_data()

# Normalize os valores de pixel
x_train, x_test = x_train / 255.0, x_test / 255.0

## Criando uma classe Callback


Você pode criar um _callback_ definindo uma classe que herda a classe base [tf.keras.callbacks.Callback](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/Callback).

A partir daí, você pode definir os métodos disponíveis para definir onde o _callback_ será executado.

Por exemplo, abaixo, você usará o método [on_epoch_end()](https://www.tensorflow.org/api_docs/python/tf)

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    '''
    Interrompe o treinamento quando a perda cai abaixo de 0,4

    Args:
      epoch (integer) - índice da época (obrigatório, mas não utilizado na definição da função abaixo)
      logs (dict) - resultados da métrica da época de treinamento
    '''

    # Check the loss
    if(logs.get('loss') < 0.4):

      # Stop if threshold is met
      print("\nA perda é menor que 0,4, portanto, o treinamento está sendo interrompido!")
      self.model.stop_training = True

# Instantiate class
callbacks = myCallback()

## Definir e compilar o modelo

Em seguida, você definirá e compilará o modelo. A arquitetura será semelhante à que você criou no laboratório anterior. Depois disso, você definirá o otimizador, a perda e as métricas que serão usadas no treinamento.

In [ ]:
# Defina o modelo
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

# Compile o modelo
model.compile(optimizer=tf.optimizers.Adam(),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])



### Treinar o modelo

Agora você está pronto para treinar o modelo. Para definir o _callback_, basta definir o parâmetro `callbacks` para a instância `myCallback` que você declarou anteriormente. Execute a célula abaixo e observe o que acontece.

In [ ]:
# Treinar o modelo com um callback
model.fit(x_train, y_train, epochs=10, callbacks=[callbacks])

Você perceberá que o treinamento não precisa concluir todas as 10 épocas.

Por ter um retorno de chamada no final de cada época, ele pode verificar os parâmetros de treinamento e comparar se ele atende ao limite que você definiu na definição da função.
> Nesse caso, ele simplesmente parará quando a perda cair abaixo de `0,40` após a época atual.

*Desafio opcional: Modifique o código para que o treinamento seja interrompido quando a métrica de acurácia exceder 60%.

Isso conclui esse exercício simples sobre callbacks!